<a href="https://colab.research.google.com/github/whkwls2653/Kaggle_JaneStreet/blob/main/colab_GPU_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
############ feature engineering + resnet

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import files
files.download('/content/gdrive/MyDrive/Kaggle/[부문 1] 데이터 명세_신경계 질환 AI데이터_CT.pdf')
files.download('/content/gdrive/MyDrive/Kaggle/[부문 1] 신경계질환_뇌혈관CT_데이터구성.xlsx')
files.download('/content/gdrive/MyDrive/Kaggle/[부문 1] 테크니컬리포트_신경계_뇌혈관질환_CT.pdf')
files.download('/content/gdrive/MyDrive/Kaggle/00000004.json')
files.download('/content/gdrive/MyDrive/Kaggle/00000001.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
!pip install tensorflow_addons

     |████████████████████████████████| 706kB 4.8MB/s 


In [2]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
import os

SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)


save=True
resume=False
Train=True
########################################여기 참조하세요!!!!!!!!!!!!!!#####################

print("reading csv input .....")
if 'train' in locals():
  print("Alreday have, skip reading")
else:
  train = pd.read_csv('/content/gdrive/MyDrive/Kaggle/janestreet/train.csv')
print("finish reading input")
########################################여기 참조하세요!!!!!!!!!!!!!!#####################



train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]



train.fillna(train.mean(),inplace=True)



train['action'] = ((train['resp'].values) > 0).astype(int) #* ((train['feature_0'].values)==-1).astype(int)

# train['action'] = (train[featrues[0]==1).astype(int)

features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X_train = train.loc[:, train.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])
for i in range(10): 
  X_train['new_feature'+str(i)]=100
  
y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

################## feature engineering#################






########################################여기 참조하세요!!!!!!!!!!!!!!#####################

def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)

        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model


batch_size = 4096
hidden_units = [160, 160, 160]
dropout_rates = [0.20, 0.20, 0.20, 0.20]
label_smoothing = 1e-2
learning_rate = 1e-3
tf.keras.backend.clear_session()
tf.random.set_seed(SEED)

checkpoint_path = "/content/gdrive/MyDrive/Kaggle/janestreet/feature_test/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
if save==True:

  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,monitor='AUC', verbose=1)





clf = create_mlp(
    len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

if resume==True:
  clf.load_weights(checkpoint_path)
if Train==True:
  clf.fit(X_train, y_train, epochs=200, batch_size=5000,callbacks=[cp_callback] )
#######################
# clf2 = create_mlp(
#     len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
#     )

# checkpoint_path = "/content/gdrive/MyDrive/Kaggle/janestreet/150hidden_250epoch/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)
# if save==True:

#   cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,monitor='AUC', verbose=1)


# clf2.fit(X_train, y_train, epochs=250, batch_size=5000,callbacks=[cp_callback] )
# #############################################################################
# clf3 = create_mlp(
#     len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
#     )

# checkpoint_path = "/content/gdrive/MyDrive/Kaggle/janestreet/150epoch_best_save_only1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)
# if save==True:

#   cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,monitor='AUC',mode='max', save_best_only=True, verbose=1)


# clf3.fit(X_train, y_train, epochs=150, batch_size=5000,callbacks=[cp_callback] )

########################################여기 참조하세요!!!!!!!!!!!!!!#####################




models = []

models.append(clf)

th = 0.502


f = np.median
models = models[-3:]
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

reading csv input .....
finish reading input


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/200


ValueError: ignored

In [8]:
print(type(X_train))
print(X_train['feature_1'])
print(X_train['feature_2'])
# for i in range(10): 
#   X_train['new_feature'+str(i)]=100
# X_train['new_feature1']=

X_train['new_feature9']=X_train['feature_1']/(X_train['feature_2']+1e-5)
print(X_train)
X_train=X_train.drop(['new_feature0','feature_1'],axis=1)
print(X_train)

<class 'pandas.core.frame.DataFrame'>
0          3.151305
2         -0.365888
3          1.514607
6         -1.008999
7         -0.391784
             ...   
1862550    1.538675
1862552    0.270380
1862584   -0.134380
1862587   -0.779554
1862595   -1.463757
Name: feature_1, Length: 1571415, dtype: float64
0          5.467693
2          0.824004
3          0.596214
6          0.242132
7          0.152261
             ...   
1862550    2.530447
1862552   -1.231874
1862584    0.160580
1862587   -0.597258
1862595   -1.107228
Name: feature_2, Length: 1571415, dtype: float64
         feature_0  feature_1  ...  new_feature8  new_feature9
0                1   3.151305  ...           100      0.576349
2               -1  -0.365888  ...           100     -0.444031
3                1   1.514607  ...           100      2.540332
6               -1  -1.008999  ...           100     -4.166965
7               -1  -0.391784  ...           100     -2.572940
...            ...        ...  ...           .

In [ ]:
a=[[1,2,3],[4,5,6]]

In [ ]:
b=(a==1)
print(b)

True


In [ ]:
print(train)
a=(train['date']==86).astype(int)* (train['weight']<1).astype(int)
print(a)
# b=(train['weight']<1).astype(int)
# print(b)
# print(a * b)

NameError: ignored